In [ ]:
conda install -c conda-forge rasterio -y

In [ ]:
conda install -c conda-forge laspy -y

In [ ]:
conda install -c conda-forge folium -y

In [ ]:
conda install -c conda-forge pandas -y

In [ ]:
conda install -c conda-forge geopandas -y

In [ ]:
conda install -c conda-forge python-pdal -y

In [ ]:
conda install -c conda-forge ipywidgets -y

In [ ]:
conda install -c conda-forge imageio -y

#### Importing all the required packages

In [3]:
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS, Transformer
import os
import pdal
from osgeo import gdal
import json
from ipywidgets import interact
from ipywidgets import widgets
import imageio
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np
import laspy
import rasterio
from rasterio import mask
from glob import glob
from shapely.geometry import box
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [4]:
data= requests.get("https://s3-us-west-2.amazonaws.com/usgs-lidar-public/AK_Coastal_2009/ept.json")

In [8]:
data_json=data.json()

In [9]:
data_json

{'bounds': [-15730544, 10937407, -19027, -15691854, 10976097, 19663],
 'boundsConforming': [-15730543, 10941090, -3, -15691855, 10972414, 638],
 'dataType': 'laszip',
 'hierarchyType': 'json',
 'points': 55711772,
 'schema': [{'name': 'X',
   'offset': -15711199,
   'scale': 0.01,
   'size': 4,
   'type': 'signed'},
  {'name': 'Y',
   'offset': 10956752,
   'scale': 0.01,
   'size': 4,
   'type': 'signed'},
  {'name': 'Z', 'offset': 318, 'scale': 0.01, 'size': 4, 'type': 'signed'},
  {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
  {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
  {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
  {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
  {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
  {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
  {'name': 'ScanAngleRank', 'size': 4, 'type': 'float'},
  {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
  {'name': 'PointSourceId', 'size': 

#### Defining the data paths

In [ ]:
PUBLIC_ACCESS_PATH= "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"





In [ ]:
## Define a function for running a pdal pipeline and returning the filename as output
    def run_pipeline(self):
        pipeline_def =  [
        {
            "bounds": self.pdal_aoi_bounds,
            "filename": self.PUBLIC_ACCESS_PATH,
            "type": "readers.ept",
            "tag": "readdata"
        },
        {
            "limits": "Classification![2:7], Classification![9:9]",
            "type": "filters.range",
            "tag": "nonoise"
        },
        {
            "filename": "spatial_subset.laz",
            "inputs": [ "nonoise" ],
            "tag": "writerslas",
            "type": "writers.las"
        },
       {
            "filename": "spatial_subset.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "inputs": [ "writerslas" ],
            "nodata": -9999,
            "output_type": "idw",
            "resolution": 1,
            "type": "writers.gdal",
            "window_size": 6
        },
        {
        "type" : "writers.text",
        "format": "geojson",
        "write_header": True
        }
    ]

In [ ]:
[
    {
        "bounds": "([-11669524.7, -11666600.8], [4776607.3, 4778714.4])",
        "filename":"https://s3-us-west-2.amazonaws.com/usgs-lidar-public/AK_Coastal_2009/ept.json",
        "type": "readers.ept"
    },
    {
        "limits": "Classification[2:2]",
        "type": "filters.range",
        "tag": "classify"
    },
    {
        "filename": "test.laz",
        "inputs": [ "classify" ],
        "tag": "writerslas",
        "type": "writers.las"
    },
    {
        "filename": "test.tif",
        "gdalopts": "tiled=yes,     compress=deflate",
        "inputs": [ "writerslas" ],
        "nodata": -9999,
        "output_type": "idw",
        "resolution": 1,
        "type": "writers.gdal",
        "window_size": 6
    }
]